In [62]:
import numpy as np
import pandas as pd
import tensorflow as tf
file = 'try_new.csv'
full_data = pd.read_csv(file)

output_feature = 'Base'
# subtract one from Base
full_data[output_feature] = full_data[output_feature] - 1
def split(df, ratio):
    msk = np.random.rand(len(df)) < ratio
    return df[msk], df[~msk]


train_data, test_data = split(full_data, 0.7)

train_data.head()


,ClusterCoordinateX,ClusterCoordinateY,DayOfWeekNum,MonthDayNum,ChunkIndex,Base
0,0,163,2,21,51,0
1,11,178,1,17,51,0
4,17,269,3,17,60,0
5,18,137,0,2,22,0
7,26,218,2,16,24,0


In [81]:
# input_features = ['x_coordinate', 'y_coordinate', 'dayofmonth', 'dayofweek', 'minchunk']
input_features = train_data.columns[:5]
get_data = tf.estimator.inputs.pandas_input_fn(train_data[input_features], train_data[output_feature], shuffle=True, num_epochs=10)



In [82]:
feature_cols = [tf.feature_column.numeric_column(k) for k in input_features]

model = tf.estimator.DNNClassifier([512, 256], feature_cols, n_classes=max(full_data[output_feature])+1, optimizer=lambda: tf.train.AdamOptimizer(learning_rate=0.0001))



INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/fh/qk54z7rs7_n_1ym5_097krp40000gn/T/tmp2tkn8fdu', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x13dcf69b0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [83]:
# start training
model.train(input_fn=get_data)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /var/folders/fh/qk54z7rs7_n_1ym5_097krp40000gn/T/tmp2tkn8fdu/model.ckpt.
INFO:tensorflow:loss = 3774.4153, step = 1
INFO:tensorflow:global_step/sec: 316.846
INFO:tensorflow:loss = 63.454216, step = 101 (0.316 sec)
INFO:tensorflow:global_step/sec: 387.311
INFO:tensorflow:loss = 149.49011, step = 201 (0.258 sec)
INFO:tensorflow:global_step/sec: 388.84
INFO:tensorflow:loss = 160.1593, step = 301 (0.257 sec)
INFO:tensorflow:global_step/sec: 358.587
INFO:tensorflow:loss = 65.429054, step = 401 (0.279 sec)
INFO:tensorflow:global_step/sec: 372.243
INFO:tensorflow:loss = 24.393747, step = 501 (0.269 sec)
INFO:tensorflow:global_step/sec: 385.579
INFO:tensorflow:loss = 71.68861, step = 601 (0.259 sec)
INFO:tensorflow:

INFO:tensorflow:loss = 236.92725, step = 8001 (0.263 sec)
INFO:tensorflow:global_step/sec: 377.426
INFO:tensorflow:loss = 189.91046, step = 8101 (0.265 sec)
INFO:tensorflow:global_step/sec: 388.405
INFO:tensorflow:loss = 153.70673, step = 8201 (0.258 sec)
INFO:tensorflow:global_step/sec: 375.758
INFO:tensorflow:loss = 68.0497, step = 8301 (0.266 sec)
INFO:tensorflow:global_step/sec: 400.016
INFO:tensorflow:loss = 58.0809, step = 8401 (0.250 sec)
INFO:tensorflow:global_step/sec: 393.592
INFO:tensorflow:loss = 130.54263, step = 8501 (0.254 sec)
INFO:tensorflow:global_step/sec: 391.382
INFO:tensorflow:loss = 157.90268, step = 8601 (0.256 sec)
INFO:tensorflow:global_step/sec: 369.727
INFO:tensorflow:loss = 255.36093, step = 8701 (0.270 sec)
INFO:tensorflow:global_step/sec: 387.914
INFO:tensorflow:loss = 323.915, step = 8801 (0.258 sec)
INFO:tensorflow:global_step/sec: 384.203
INFO:tensorflow:loss = 212.54785, step = 8901 (0.260 sec)
INFO:tensorflow:global_step/sec: 390.071
INFO:tensorflow:

INFO:tensorflow:global_step/sec: 363.613
INFO:tensorflow:loss = 132.61877, step = 16301 (0.275 sec)
INFO:tensorflow:global_step/sec: 365.671
INFO:tensorflow:loss = 184.55225, step = 16401 (0.274 sec)
INFO:tensorflow:global_step/sec: 372.072
INFO:tensorflow:loss = 62.40824, step = 16501 (0.269 sec)
INFO:tensorflow:global_step/sec: 339.347
INFO:tensorflow:loss = 83.828964, step = 16601 (0.295 sec)
INFO:tensorflow:global_step/sec: 372.261
INFO:tensorflow:loss = 244.61928, step = 16701 (0.269 sec)
INFO:tensorflow:global_step/sec: 370.065
INFO:tensorflow:loss = 214.72171, step = 16801 (0.270 sec)
INFO:tensorflow:global_step/sec: 370.266
INFO:tensorflow:loss = 204.63739, step = 16901 (0.270 sec)
INFO:tensorflow:global_step/sec: 363.777
INFO:tensorflow:loss = 262.76855, step = 17001 (0.275 sec)
INFO:tensorflow:global_step/sec: 350.274
INFO:tensorflow:loss = 247.76636, step = 17101 (0.285 sec)
INFO:tensorflow:global_step/sec: 370.65
INFO:tensorflow:loss = 165.37009, step = 17201 (0.270 sec)
IN

INFO:tensorflow:loss = 239.63278, step = 24501 (0.292 sec)
INFO:tensorflow:global_step/sec: 332.845
INFO:tensorflow:loss = 26.606268, step = 24601 (0.300 sec)
INFO:tensorflow:global_step/sec: 337.503
INFO:tensorflow:loss = 58.860157, step = 24701 (0.296 sec)
INFO:tensorflow:global_step/sec: 317.822
INFO:tensorflow:loss = 18.259136, step = 24801 (0.315 sec)
INFO:tensorflow:global_step/sec: 309.18
INFO:tensorflow:loss = 81.856964, step = 24901 (0.323 sec)
INFO:tensorflow:global_step/sec: 346.166
INFO:tensorflow:loss = 257.89066, step = 25001 (0.289 sec)
INFO:tensorflow:global_step/sec: 336.788
INFO:tensorflow:loss = 136.11499, step = 25101 (0.297 sec)
INFO:tensorflow:global_step/sec: 326.787
INFO:tensorflow:loss = 33.127075, step = 25201 (0.306 sec)
INFO:tensorflow:global_step/sec: 300.183
INFO:tensorflow:loss = 13.606888, step = 25301 (0.333 sec)
INFO:tensorflow:global_step/sec: 317.659
INFO:tensorflow:loss = 155.1738, step = 25401 (0.315 sec)
INFO:tensorflow:global_step/sec: 322.731
IN

INFO:tensorflow:global_step/sec: 303.472
INFO:tensorflow:loss = 42.543312, step = 32801 (0.330 sec)
INFO:tensorflow:global_step/sec: 312.587
INFO:tensorflow:loss = 92.08215, step = 32901 (0.320 sec)
INFO:tensorflow:global_step/sec: 330.23
INFO:tensorflow:loss = 172.83371, step = 33001 (0.302 sec)
INFO:tensorflow:global_step/sec: 345.883
INFO:tensorflow:loss = 73.634476, step = 33101 (0.289 sec)
INFO:tensorflow:global_step/sec: 321.463
INFO:tensorflow:loss = 202.67908, step = 33201 (0.311 sec)
INFO:tensorflow:global_step/sec: 346.203
INFO:tensorflow:loss = 183.91846, step = 33301 (0.289 sec)
INFO:tensorflow:global_step/sec: 345.88
INFO:tensorflow:loss = 45.26635, step = 33401 (0.289 sec)
INFO:tensorflow:global_step/sec: 351.01
INFO:tensorflow:loss = 33.442318, step = 33501 (0.285 sec)
INFO:tensorflow:global_step/sec: 330.528
INFO:tensorflow:loss = 90.15384, step = 33601 (0.302 sec)
INFO:tensorflow:global_step/sec: 344.21
INFO:tensorflow:loss = 53.040104, step = 33701 (0.291 sec)
INFO:te

...

INFO:tensorflow:loss = 230.9885, step = 74001 (0.278 sec)
INFO:tensorflow:global_step/sec: 366.826
INFO:tensorflow:loss = 212.16434, step = 74101 (0.273 sec)
INFO:tensorflow:global_step/sec: 358.997
INFO:tensorflow:loss = 242.20418, step = 74201 (0.279 sec)
INFO:tensorflow:global_step/sec: 362.52
INFO:tensorflow:loss = 165.13358, step = 74301 (0.276 sec)
INFO:tensorflow:global_step/sec: 363.867
INFO:tensorflow:loss = 63.215652, step = 74401 (0.275 sec)
INFO:tensorflow:global_step/sec: 368.301
INFO:tensorflow:loss = 52.60075, step = 74501 (0.271 sec)
INFO:tensorflow:global_step/sec: 365.386
INFO:tensorflow:loss = 92.89211, step = 74601 (0.274 sec)
INFO:tensorflow:global_step/sec: 338.742
INFO:tensorflow:loss = 222.33394, step = 74701 (0.295 sec)
INFO:tensorflow:global_step/sec: 352.396
INFO:tensorflow:loss = 172.01422, step = 74801 (0.284 sec)
INFO:tensorflow:global_step/sec: 360.475
INFO:tensorflow:loss = 249.57022, step = 74901 (0.277 sec)
INFO:tensorflow:global_step/sec: 355.887
INFO

INFO:tensorflow:global_step/sec: 381.975
INFO:tensorflow:loss = 224.41734, step = 82301 (0.262 sec)
INFO:tensorflow:global_step/sec: 379.655
INFO:tensorflow:loss = 217.27667, step = 82401 (0.263 sec)
INFO:tensorflow:global_step/sec: 385.125
INFO:tensorflow:loss = 226.59932, step = 82501 (0.260 sec)
INFO:tensorflow:global_step/sec: 354.964
INFO:tensorflow:loss = 226.882, step = 82601 (0.282 sec)
INFO:tensorflow:global_step/sec: 386.678
INFO:tensorflow:loss = 243.53741, step = 82701 (0.259 sec)
INFO:tensorflow:global_step/sec: 385.281
INFO:tensorflow:loss = 280.46252, step = 82801 (0.260 sec)
INFO:tensorflow:global_step/sec: 389.017
INFO:tensorflow:loss = 227.54008, step = 82901 (0.257 sec)
INFO:tensorflow:global_step/sec: 384.261
INFO:tensorflow:loss = 39.015553, step = 83001 (0.260 sec)
INFO:tensorflow:global_step/sec: 384.904
INFO:tensorflow:loss = 271.5228, step = 83101 (0.260 sec)
INFO:tensorflow:global_step/sec: 384.138
INFO:tensorflow:loss = 282.81122, step = 83201 (0.260 sec)
INF

INFO:tensorflow:loss = 21.044106, step = 90501 (0.271 sec)
INFO:tensorflow:global_step/sec: 389.835
INFO:tensorflow:loss = 71.41246, step = 90601 (0.256 sec)
INFO:tensorflow:global_step/sec: 391.497
INFO:tensorflow:loss = 50.264465, step = 90701 (0.256 sec)
INFO:tensorflow:global_step/sec: 390.712
INFO:tensorflow:loss = 32.658264, step = 90801 (0.256 sec)
INFO:tensorflow:global_step/sec: 371.872
INFO:tensorflow:loss = 37.498596, step = 90901 (0.269 sec)
INFO:tensorflow:global_step/sec: 386.133
INFO:tensorflow:loss = 57.999683, step = 91001 (0.259 sec)
INFO:tensorflow:global_step/sec: 378.353
INFO:tensorflow:loss = 40.53547, step = 91101 (0.264 sec)
INFO:tensorflow:global_step/sec: 372.301
INFO:tensorflow:loss = 35.946693, step = 91201 (0.269 sec)
INFO:tensorflow:global_step/sec: 389.652
INFO:tensorflow:loss = 28.039448, step = 91301 (0.257 sec)
INFO:tensorflow:global_step/sec: 385.282
INFO:tensorflow:loss = 35.275505, step = 91401 (0.259 sec)
INFO:tensorflow:global_step/sec: 385.024
IN

INFO:tensorflow:global_step/sec: 338.186
INFO:tensorflow:loss = 54.974247, step = 98801 (0.296 sec)
INFO:tensorflow:global_step/sec: 345.661
INFO:tensorflow:loss = 117.62331, step = 98901 (0.290 sec)
INFO:tensorflow:global_step/sec: 364.291
INFO:tensorflow:loss = 252.6345, step = 99001 (0.274 sec)
INFO:tensorflow:global_step/sec: 360.255
INFO:tensorflow:loss = 241.47693, step = 99101 (0.278 sec)
INFO:tensorflow:global_step/sec: 365.253
INFO:tensorflow:loss = 87.39822, step = 99201 (0.273 sec)
INFO:tensorflow:global_step/sec: 355.101
INFO:tensorflow:loss = 89.877625, step = 99301 (0.282 sec)
INFO:tensorflow:global_step/sec: 334.728
INFO:tensorflow:loss = 63.696445, step = 99401 (0.299 sec)
INFO:tensorflow:global_step/sec: 351.853
INFO:tensorflow:loss = 156.06177, step = 99501 (0.284 sec)
INFO:tensorflow:global_step/sec: 331.839
INFO:tensorflow:loss = 140.5843, step = 99601 (0.301 sec)
INFO:tensorflow:global_step/sec: 328.353
INFO:tensorflow:loss = 165.12791, step = 99701 (0.305 sec)
INF

INFO:tensorflow:global_step/sec: 385.746
INFO:tensorflow:loss = 64.09622, step = 107001 (0.259 sec)
INFO:tensorflow:global_step/sec: 385.253
INFO:tensorflow:loss = 96.26497, step = 107101 (0.260 sec)
INFO:tensorflow:global_step/sec: 384.391
INFO:tensorflow:loss = 146.96945, step = 107201 (0.260 sec)
INFO:tensorflow:global_step/sec: 350.764
INFO:tensorflow:loss = 168.84903, step = 107301 (0.285 sec)
INFO:tensorflow:global_step/sec: 362.083
INFO:tensorflow:loss = 203.57079, step = 107401 (0.276 sec)
INFO:tensorflow:global_step/sec: 358.775
INFO:tensorflow:loss = 281.36316, step = 107501 (0.279 sec)
INFO:tensorflow:global_step/sec: 331.066
INFO:tensorflow:loss = 212.25742, step = 107601 (0.302 sec)
INFO:tensorflow:global_step/sec: 378.659
INFO:tensorflow:loss = 189.61743, step = 107701 (0.264 sec)
INFO:tensorflow:global_step/sec: 345.375
INFO:tensorflow:loss = 229.53564, step = 107801 (0.289 sec)
INFO:tensorflow:global_step/sec: 367.953
INFO:tensorflow:loss = 157.29208, step = 107901 (0.2

In [78]:
# evaluate metrics
get_test_data = tf.estimator.inputs.pandas_input_fn(test_data[input_features], test_data[output_feature], shuffle=True)
model.evaluate(input_fn=get_test_data)


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-06-06T06:14:07Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/fh/qk54z7rs7_n_1ym5_097krp40000gn/T/tmpzfmk60oz/model.ckpt-56010
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-06-06-06:14:15
INFO:tensorflow:Saving dict for global step 56010: accuracy = 0.5997057, average_loss = 1.4055748, global_step = 56010, loss = 179.8854
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 56010: /var/folders/fh/qk54z7rs7_n_1ym5_097krp40000gn/T/tmpzfmk60oz/model.ckpt-56010


{'accuracy': 0.5997057,
 'average_loss': 1.4055748,
 'loss': 179.8854,
 'global_step': 56010}

In [80]:
# predict
prediction_data = test_data[5:8]
print(prediction_data.head())

get_prediction_data = tf.estimator.inputs.pandas_input_fn(prediction_data[input_features], prediction_data[output_feature], shuffle=False, num_epochs=1)
list(model.predict(input_fn=get_prediction_data))

    ClusterCoordinateX  ClusterCoordinateY  DayOfWeekNum  MonthDayNum  \
15                  36                 165             4           19   
16                  38                 132             3           10   
22                  39                 221             4           22   

    ChunkIndex  Base  
15          58     0  
16          72     0  
22          56     0  
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/fh/qk54z7rs7_n_1ym5_097krp40000gn/T/tmpzfmk60oz/model.ckpt-56010
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


[{'logits': array([  5.200681 ,   4.121735 ,   2.5672936,   0.5282359,  -0.6152709,
          -1.6932842,  -3.6285505,  -3.2394896,  -6.5522223,  -6.7849503,
          -5.815725 ,  -4.9695363,  -6.8484   ,  -7.8592367,  -7.9865947,
          -7.0638967, -10.532853 ,  -9.180073 , -11.860085 , -11.416455 ,
         -10.74991  , -11.420926 , -10.722576 , -10.52589  , -14.5951605,
         -13.328194 , -16.523994 , -14.433364 , -15.433117 , -15.02038  ,
         -18.617886 , -18.026608 , -15.98516  , -18.927778 , -20.464872 ,
         -20.292479 , -18.602755 , -19.558033 , -21.163078 , -19.208973 ,
         -21.783129 , -26.943079 , -20.222704 , -26.182396 , -20.461061 ,
         -19.480263 , -28.558489 , -23.645895 , -24.069824 , -29.369898 ,
         -33.467335 , -31.330029 , -28.927286 , -29.368849 , -30.55372  ,
         -27.231928 , -20.918932 , -28.307497 , -29.598791 , -27.188847 ,
         -21.73226  , -28.781767 , -21.39126  , -27.57989  , -29.505196 ,
         -29.245588 , -26.86